<a href="https://colab.research.google.com/github/shardulburde/Exotic-Options-Pricing/blob/main/BarrierOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
xls = pd.ExcelFile('/content/drive/MyDrive/IndusInd/Barrier_Data.xlsx')
df = pd.read_excel(xls, 'Sheet1')
df1 = pd.read_excel(xls, 'Sheet2')
df2 = pd.read_excel(xls, 'Sheet3')
 
from scipy.stats import norm
 
df

,Unnamed: 0,Up & Out,Down & Out,Up & In,Down & In,Unnamed: 5,Tenor,Decimal
0,Strike,75.0000,75.0000,78.0000,100.0000,NaN,1W,0.020833
1,Spot,73.2750,73.2750,73.2750,73.2750,NaN,2W,0.041667
2,Barrier,80.0000,70.0000,76.0000,97.0000,NaN,3W,0.062500
3,Tenor(years),0.5000,0.5000,0.5000,0.5000,NaN,1M,0.083333
4,INR Rate(%),0.0445,0.0445,0.0445,0.0445,NaN,2M,0.166667
5,USD Rate(%),0.0006,0.0006,0.0006,0.0006,NaN,3M,0.250000
6,Vol(%),5.5600,24.0000,5.7500,24.0000,NaN,4M,0.333333
7,TimeStep(days),1.0000,1.0000,1.0000,1.0000,NaN,5M,0.416667
8,NumSim,10000.0000,10000.0000,10000.0000,10000.0000,NaN,6M,0.500000
9,NaN,NaN,NaN,NaN,NaN,NaN,9M,0.583333


In [ ]:
tenor=df._get_value(3,1,True)
x=df2.loc[df2['Time'] == tenor,'50D Call']
df2


,Unnamed: 0,Time,10D Call,25D Call,50D Call,75D Call,90D Call
0,1W,0.020833,0.053925,0.049825,0.046500,0.045275,0.045675
1,2W,0.041667,0.053937,0.049900,0.046650,0.045400,0.045812
2,3W,0.062500,0.053887,0.049862,0.046575,0.045338,0.045762
3,1M,0.083333,0.056200,0.050787,0.046550,0.045463,0.046650
4,2M,0.166667,0.060075,0.053762,0.048950,0.047988,0.049525
5,3M,0.250000,0.064900,0.057612,0.052250,0.051338,0.053250
6,4M,0.333333,0.067538,0.059675,0.054025,0.053375,0.055813
7,5M,0.416667,0.070395,0.061270,0.055185,0.054800,0.058015
8,6M,0.500000,0.071625,0.062075,0.055650,0.055425,0.059175
9,9M,0.583333,0.074325,0.064200,0.057450,0.057500,0.061725


In [ ]:
from scipy import interpolate

def f(x):
    y1=float(df2.loc[df2['Time'] == tenor,'10D Call'])
    y2=float(df2.loc[df2['Time'] == tenor,'25D Call'])
    y3=float(df2.loc[df2['Time'] == tenor,'50D Call'])
    y4=float(df2.loc[df2['Time'] == tenor,'75D Call'])
    y5=float(df2.loc[df2['Time'] == tenor,'90D Call'])

    x_points = [0.1, 0.25, 0.5, 0.75, 0.9]
    y_points = [y1,y2,y3,y4,y5]

    tck = interpolate.splrep(x_points, y_points)
    return interpolate.splev(x, tck)

print(f(0.3))

0.06003841025641026


In [ ]:
#Making Vol Lookup Function
import numpy as np
import math
import scipy
from scipy.stats import norm

#Vol for Up Out
def VolUpOut():
  
  tenor=df._get_value(3,1,True)
  rate=df._get_value(4,1,True)-df._get_value(5,1,True)
  strike=df._get_value(0,1,True)
  spot=df._get_value(1,1,True)

  x=df2.loc[df2['Time'] == tenor,'50D Call']
  vol = float(x)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-vol*vol/2)*tenor)/(vol*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    vol=f(delta)

  return float(vol)


#Vol for Down Out
def VolDownOut():
  
  tenor=df._get_value(3,2,True)
  rate=df._get_value(4,2,True)-df._get_value(5,2,True)
  strike=df._get_value(0,2,True)
  spot=df._get_value(1,2,True)

  x=df2.loc[df2['Time'] == tenor,'50D Call']
  vol = float(x)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-vol*vol/2)*tenor)/(vol*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    vol=f(delta)

  return float(vol)
  
  
#Vol for Up In
def VolUpIn():
  
  tenor=df._get_value(3,3,True)
  rate=df._get_value(4,3,True)-df._get_value(5,3,True)
  strike=df._get_value(0,3,True)
  spot=df._get_value(1,3,True)

  x=df2.loc[df2['Time'] == tenor,'50D Call']
  vol = float(x)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-vol*vol/2)*tenor)/(vol*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    vol=f(delta)

  return float(vol)

#Vol for Down In
def VolDownIn():
  
  tenor=df._get_value(3,4,True)
  rate=df._get_value(4,4,True)-df._get_value(5,4,True)
  strike=df._get_value(0,4,True)
  spot=df._get_value(1,4,True)

  x=df2.loc[df2['Time'] == tenor,'50D Call']
  vol = float(x)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-vol*vol/2)*tenor)/(vol*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    vol=f(delta)

  return float(vol)
  
  




In [ ]:
import math
import numpy as np

#Creating Knock Out Pricing Functions

#Up and Out

def UpOut(strike, spot, tenor, INRr, USDr, vol, t, numsim, bar):
  
  
  rate = (INRr)/100-(USDr)/100
  
  sum = 0.0
  vol=VolUpOut()

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd > bar :
        fwd=0
  
    payoff = max(fwd - strike, 0)
    sum = sum + payoff

  Up_Out_Call=(sum / numsim) * math.exp(-rate * tenor)

  sum = 0.0
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd > bar :
        fwd=0
  
    if fwd == 0: 
      payoff = 0
        
    else:
      payoff = max(strike - fwd, 0)
            
         
    sum = sum + payoff

  Up_Out_Put=(sum / numsim) * math.exp(-rate * tenor)

         
  return Up_Out_Call,Up_Out_Put




#Down and Out

def DownOut(strike, spot, tenor, INRr, USDr, vol, t, numsim, bar):
  
 


  rate = np.log(1+(INRr)/100-(USDr)/100) 
  sum = 0.0
  vol=VolDownOut()

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd < bar :
        fwd=0
  
    payoff = max(fwd - strike, 0)
    sum = sum + payoff

  Down_Out_Call=(sum / numsim) * math.exp(-rate * tenor)

  sum = 0.0
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd < bar :
        fwd=0
  
    if fwd == 0: 
      payoff = 0
        
    else:
      payoff = max(strike - fwd, 0)
            
         
    sum = sum + payoff

  Down_Out_Put=(sum / numsim) * math.exp(-rate * tenor)

         
  return Down_Out_Call,Down_Out_Put


#Creating Knock In Pricing Functions

#Up and In

def UpIn(strike, spot, tenor, INRr, USDr, vol, t, numsim, bar):
  
  
  rate = np.log(1+(INRr)/100-(USDr)/100)
  
  sum = 0.0
  vol=VolUpIn()

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd > bar :
        for k in range(n-j):
          fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

        payoff = max(fwd - strike, 0)
        sum = sum + payoff
        
        break

    

  Up_In_Call=(sum / numsim) * math.exp(-rate * tenor)

  sum = 0
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd > bar :
        for k in range(n-j):
          fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

        payoff = max(strike-fwd, 0)

        sum = sum + payoff
        break

      

  Up_In_Put=(sum / numsim) * math.exp(-rate * tenor)



         
  return Up_In_Call,Up_In_Put 

#Down and In

def DownIn(strike, spot, tenor, INRr, USDr, vol, t, numsim, bar):
  
  
  rate = np.log(1+(INRr)/100-(USDr)/100)
  
  sum = 0.0
  vol=VolDownIn()

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd < bar :
        for k in range(n-j):
          fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

        payoff = max(fwd - strike, 0)
        sum = sum + payoff
        break

      

  Down_In_Call=(sum / numsim) * math.exp(-rate * tenor)

  sum = 0
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

      if fwd < bar :
        for k in range(n-j):
          fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

        payoff = max(strike-fwd, 0)

        sum = sum + payoff
        break

    

  Down_In_Put=(sum / numsim) * math.exp(-rate * tenor)



         
  return Down_In_Call,Down_In_Put 
df


,Unnamed: 0,Up & Out,Down & Out,Up & In,Down & In,Unnamed: 5,Tenor,Decimal
0,Strike,75.0000,75.0000,78.0000,100.0000,NaN,1W,0.020833
1,Spot,73.2750,73.2750,73.2750,73.2750,NaN,2W,0.041667
2,Barrier,80.0000,70.0000,76.0000,97.0000,NaN,3W,0.062500
3,Tenor(years),0.5000,0.5000,0.5000,0.5000,NaN,1M,0.083333
4,INR Rate(%),0.0445,0.0445,0.0445,0.0445,NaN,2M,0.166667
5,USD Rate(%),0.0006,0.0006,0.0006,0.0006,NaN,3M,0.250000
6,Vol(%),5.5600,24.0000,5.7500,24.0000,NaN,4M,0.333333
7,TimeStep(days),1.0000,1.0000,1.0000,1.0000,NaN,5M,0.416667
8,NumSim,10000.0000,10000.0000,10000.0000,10000.0000,NaN,6M,0.500000
9,NaN,NaN,NaN,NaN,NaN,NaN,9M,0.583333


Up and Out


In [53]:
print('Call and Put Price of Up and Out is',UpOut(df._get_value(0, 'Up & Out'),df._get_value(1, 'Up & Out'),df._get_value(3, 'Up & Out'),df._get_value(4, 'Up & Out'),df._get_value(5, 'Up & Out'),df._get_value(6, 'Up & Out'),df._get_value(7, 'Up & Out'),int(df._get_value(8, 'Up & Out')),df._get_value(2, 'Up & Out')))

Call and Put Price of Up and Out is (0.3892785418721344, 2.202796908654647)


Down And Out

In [ ]:
print('Call and Put Price of Down and Out is',DownOut(df._get_value(0, 'Down & Out'),df._get_value(1, 'Down & Out'),df._get_value(3, 'Down & Out'),df._get_value(4, 'Down & Out'),df._get_value(5, 'Down & Out'),df._get_value(6, 'Down & Out'),df._get_value(7, 'Down & Out'),int(df._get_value(8, 'Down & Out')),df._get_value(2, 'Down & Out')))

Call and Put Price of Down and Out is (3.9778431995136234, 0.015158722019103963)


Up and In

In [ ]:
print('Call and Put Price of Up and In is',UpIn(df._get_value(0, 'Up & In'),df._get_value(1, 'Up & In'),df._get_value(3, 'Up & In'),df._get_value(4, 'Up & In'),df._get_value(5, 'Up & In'),df._get_value(6, 'Up & In'),df._get_value(7, 'Up & In'),int(df._get_value(8, 'Up & In')),df._get_value(2, 'Up & In')))

Call and Put Price of Up and In is (3.273999885702771, 0.8160797549527762)


Down and In

In [ ]:
print('Call and Put Price of Down and In is',UpIn(df._get_value(0, 'Down & In'),df._get_value(1, 'Down & In'),df._get_value(3, 'Down & In'),df._get_value(4, 'Down & In'),df._get_value(5, 'Down & In'),df._get_value(6, 'Down & In'),df._get_value(7, 'Down & In'),int(df._get_value(8, 'Down & In')),df._get_value(2, 'Down & In')))

Call and Put Price of Down and In is (1.6663353899835007, 1.765076944948419)
